# IML - Project
## Hossein Anjidani & Zahra Maleki


In [ ]:
name1 = "Hossein Anjidani"
name2 = "Zahra Maleki"
st1 = 400100746
st2 = 400110009
print(name1, st1)
print(name2, st2)

Hossein Anjidani 400100746
Zahra Maleki 400110009


# Private Training - Question 4

In [ ]:
# install necessary components
!pip install opacus

In [ ]:
# import necessary libs
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from opacus import PrivacyEngine
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/UNI/Sem 6/ML/Project'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Included python file
class CIFAR10Classifier(nn.Module):
  def __init__(self):
    super(CIFAR10Classifier, self).__init__()
    self.conv1 = nn.Conv2d(3, 16, 3, 1)
    self.conv2 = nn.Conv2d(16, 32, 3, 1)
    self.dropout1 = nn.Dropout2d(0.25)
    self.dropout2 = nn.Dropout2d(0.5)
    self.fc1 = nn.Linear(6272, 64)
    self.fc2 = nn.Linear(64, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = self.dropout1(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    return x


In [ ]:
# include CIFAR10 Dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Load the CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split the training data into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Print the number of training, validation, and test samples
print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(val_dataset))
print("Number of test samples:", len(test_dataset))

Files already downloaded and verified
Files already downloaded and verified
Number of training samples: 40000
Number of validation samples: 10000
Number of test samples: 10000


In [ ]:
# Training function
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Validation function
def validate(model, device, val_loader, criterion):
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= len(val_loader.dataset)
    print(f'\nValidation set: Average loss: {val_loss:.4f}, Accuracy: {correct}/{len(val_loader.dataset)}'
          f' ({100. * correct / len(val_loader.dataset):.0f}%)\n')


# Test function (Privacy)
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)}'
          f' ({100. * correct / len(test_loader.dataset):.0f}%)\n')
    return 100. * correct / len(test_loader.dataset)

In [ ]:
# Instantiate the model, define the loss function and the optimizer
model = CIFAR10Classifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

In [ ]:
# Set device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training and validation loop
num_epochs = 15
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, criterion, epoch)
    validate(model, device, val_loader, criterion)

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Train Epoch: 1 [0/40000 (0%)]	Loss: 2.293126
Train Epoch: 1 [6400/40000 (16%)]	Loss: 1.768904
Train Epoch: 1 [12800/40000 (32%)]	Loss: 1.781297
Train Epoch: 1 [19200/40000 (48%)]	Loss: 1.839950
Train Epoch: 1 [25600/40000 (64%)]	Loss: 1.517123
Train Epoch: 1 [32000/40000 (80%)]	Loss: 1.633967
Train Epoch: 1 [38400/40000 (96%)]	Loss: 1.727788

Validation set: Average loss: 0.0216, Accuracy: 5193/10000 (52%)

Train Epoch: 2 [0/40000 (0%)]	Loss: 1.537562
Train Epoch: 2 [6400/40000 (16%)]	Loss: 1.423624
Train Epoch: 2 [12800/40000 (32%)]	Loss: 1.391905
Train Epoch: 2 [19200/40000 (48%)]	Loss: 1.475591
Train Epoch: 2 [25600/40000 (64%)]	Loss: 1.458956
Train Epoch: 2 [32000/40000 (80%)]	Loss: 1.433999
Train Epoch: 2 [38400/40000 (96%)]	Loss: 1.397605

Validation set: Average loss: 0.0188, Accuracy: 5771/10000 (58%)

Train Epoch: 3 [0/40000 (0%)]	Loss: 1.255708
Train Epoch: 3 [6400/40000 (16%)]	Loss: 1.602806
Train Epoch: 3 [12800/40000 (32%)]	Loss: 0.963266
Train Epoch: 3 [19200/40000 (48%)]

In [ ]:
# Save the trained model
torch.save(model.state_dict(), "cifar10_baseline_model.pth")
baseline_accuracy = test(model, device, test_loader)


Test set: Average loss: 0.0153, Accuracy: 6579/10000 (66%)



# Private Training - Question 5

In [ ]:
from opacus.validators import ModuleValidator
model_pr = CIFAR10Classifier()


errors = ModuleValidator.validate(model_pr, strict=False)
errors[-5:]

model_pr = ModuleValidator.fix(model_pr)
ModuleValidator.validate(model_pr, strict=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model_pr.to(device)
criterion = nn.CrossEntropyLoss()
LR = 0.001
optimizer = optim.RMSprop(model_pr.parameters(), lr=LR)


In [ ]:
from opacus import PrivacyEngine
MAX_GRAD_NORM = 1.2
EPSILON = 50.0
DELTA = 1e-5
EPOCHS = 15
BATCH_SIZE = 512
MAX_PHYSICAL_BATCH_SIZE = 128
LR = 1e-3
privacy_engine = PrivacyEngine()

model_pr, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model_pr,
    optimizer=optimizer,
    data_loader=train_loader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)

print(f"Using sigma={optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")

/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the smallest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/prv/prvs.py:52: RuntimeWarning: overflow encountered in exp
  z = np.log(np.where(t > np.log(1 - q), (np.exp(t) + q - 1) / q, 1))
/usr/local/lib/python3.10/dist-pac

Using sigma=0.3096771240234375 and C=1.2


In [ ]:
def accuracy(preds, labels):
    return (preds == labels).mean()

import numpy as np
from opacus.utils.batch_memory_manager import BatchMemoryManager


def train(model, train_loader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    top1_acc = []

    with BatchMemoryManager(
        data_loader=train_loader,
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE,
        optimizer=optimizer
    ) as memory_safe_data_loader:

        for i, (images, target) in enumerate(memory_safe_data_loader):
            optimizer.zero_grad()
            images = images.to(device)
            target = target.to(device)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()

            # measure accuracy and record loss
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

            loss.backward()
            optimizer.step()

            if (i+1) % 200 == 0:
                epsilon = privacy_engine.get_epsilon(DELTA)
                print(
                    f"\tTrain Epoch: {epoch} \t"
                    f"Loss: {np.mean(losses):.6f} "
                    f"Acc@1: {np.mean(top1_acc) * 100:.6f} "
                    f"(ε = {epsilon:.2f}, δ = {DELTA})"
                )
def test(model, test_loader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []

    with torch.no_grad():
        for images, target in test_loader:
            images = images.to(device)
            target = target.to(device)

            output = model(images)
            loss = criterion(output, target)
            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

    top1_avg = np.mean(top1_acc)

    print(
        f"\tTest set:"
        f"Loss: {np.mean(losses):.6f} "
        f"Acc: {top1_avg * 100:.6f} "
    )
    return np.mean(top1_acc)

In [ ]:
from tqdm.notebook import tqdm

for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
    train(model_pr, train_loader, optimizer, epoch + 1, device)

Epoch:   0%|          | 0/15 [00:00<?, ?epoch/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


	Train Epoch: 1 	Loss: 2.116126 Acc@1: 22.440465 (ε = 13.52, δ = 1e-05)
	Train Epoch: 1 	Loss: 2.071803 Acc@1: 24.792369 (ε = 15.89, δ = 1e-05)
	Train Epoch: 1 	Loss: 2.049935 Acc@1: 26.383371 (ε = 17.67, δ = 1e-05)
	Train Epoch: 2 	Loss: 1.984687 Acc@1: 31.004076 (ε = 19.35, δ = 1e-05)
	Train Epoch: 2 	Loss: 1.975376 Acc@1: 31.810454 (ε = 20.66, δ = 1e-05)
	Train Epoch: 2 	Loss: 1.968866 Acc@1: 32.324591 (ε = 21.86, δ = 1e-05)
	Train Epoch: 3 	Loss: 1.943159 Acc@1: 34.529360 (ε = 23.11, δ = 1e-05)
	Train Epoch: 3 	Loss: 1.942366 Acc@1: 34.611096 (ε = 24.14, δ = 1e-05)
	Train Epoch: 3 	Loss: 1.942284 Acc@1: 34.700264 (ε = 25.13, δ = 1e-05)
	Train Epoch: 4 	Loss: 1.926930 Acc@1: 36.606448 (ε = 26.19, δ = 1e-05)
	Train Epoch: 4 	Loss: 1.925441 Acc@1: 36.092788 (ε = 27.09, δ = 1e-05)
	Train Epoch: 4 	Loss: 1.934675 Acc@1: 36.196113 (ε = 27.95, δ = 1e-05)
	Train Epoch: 5 	Loss: 1.934931 Acc@1: 37.044004 (ε = 28.90, δ = 1e-05)
	Train Epoch: 5 	Loss: 1.935749 Acc@1: 36.998369 (ε = 29.71, δ =

In [ ]:
top1_acc = test(model_pr, test_loader, device)

	Test set:Loss: 1.533106 Acc: 47.133758 


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
# Load previously trained models (replace with actual model paths)
baseline_model = model
private_model = model_pr

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
baseline_model.to(device)
private_model.to(device)

# Function to get outputs and labels
def get_outputs_and_labels(model, loader, device, label):
    model.eval()
    outputs = []
    labels = []
    with torch.no_grad():
        for data, _ in loader:
            data = data.to(device)
            output = model(data)
            outputs.append(output.cpu().numpy())
            labels.extend([label] * data.size(0))
    outputs = np.concatenate(outputs, axis=0)
    return outputs, labels

# Generate membership labels and outputs for attacker models
# Seen data: label 1
seen_outputs_baseline, seen_labels_baseline = get_outputs_and_labels(baseline_model, train_loader, device, 1)
seen_outputs_private, seen_labels_private = get_outputs_and_labels(private_model, train_loader, device, 1)

# Unseen data: label 0
unseen_outputs_baseline, unseen_labels_baseline = get_outputs_and_labels(baseline_model, val_loader, device, 0)
unseen_outputs_baseline_test, unseen_labels_baseline_test = get_outputs_and_labels(baseline_model, test_loader, device, 0)

unseen_outputs_private, unseen_labels_private = get_outputs_and_labels(private_model, val_loader, device, 0)
unseen_outputs_private_test, unseen_labels_private_test = get_outputs_and_labels(private_model, test_loader, device, 0)

# Combine seen and unseen data
baseline_outputs = np.vstack([seen_outputs_baseline, unseen_outputs_baseline, unseen_outputs_baseline_test])
baseline_labels = seen_labels_baseline + unseen_labels_baseline + unseen_labels_baseline_test
private_outputs = np.vstack([seen_outputs_private, unseen_outputs_private, unseen_outputs_private_test])
private_labels = seen_labels_private + unseen_labels_private + unseen_labels_private_test

In [ ]:
# Define and train attacker models
class AttackerModel(nn.Module):
    def __init__(self):
        super(AttackerModel, self).__init__()
        self.fc1 = nn.Linear(10, 50)
        self.fc2 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

attacker_baseline = AttackerModel().to(device)
attacker_private = AttackerModel().to(device)

criterion = nn.CrossEntropyLoss()
optimizer_attacker_baseline = optim.Adam(attacker_baseline.parameters(), lr=0.001)
optimizer_attacker_private = optim.Adam(attacker_private.parameters(), lr=0.001)

def train_attacker(model, data, labels, optimizer, epochs=10):
    model.train()
    data = torch.tensor(data, dtype=torch.float32).to(device)
    labels = torch.tensor(labels, dtype=torch.long).to(device)
    dataset = torch.utils.data.TensorDataset(data, labels)
    loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

    for epoch in range(epochs):
        for inputs, targets in loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

train_attacker(attacker_baseline, baseline_outputs, baseline_labels, optimizer_attacker_baseline)
train_attacker(attacker_private, private_outputs, private_labels, optimizer_attacker_private)

# Evaluate attacker models
def evaluate_attacker(model, data, labels):
    model.eval()
    data = torch.tensor(data, dtype=torch.float32).to(device)
    labels = torch.tensor(labels, dtype=torch.long).to(device)
    dataset = torch.utils.data.TensorDataset(data, labels)
    loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=False)

    correct = 0
    with torch.no_grad():
        for inputs, targets in loader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == targets).item()
    accuracy = correct / len(labels)
    return accuracy

attacker_baseline_acc = evaluate_attacker(attacker_baseline, baseline_outputs, baseline_labels)
attacker_private_acc = evaluate_attacker(attacker_private, private_outputs, private_labels)

print(f'Attacker Baseline Model Accuracy: {attacker_baseline_acc * 100:.2f}%')
print(f'Attacker Private Model Accuracy: {attacker_private_acc * 100:.2f}%')

Epoch 1, Loss: 0.5299001336097717
Epoch 2, Loss: 0.551960825920105
Epoch 3, Loss: 0.6493751406669617
Epoch 4, Loss: 0.5085877180099487
Epoch 5, Loss: 0.663835883140564
Epoch 6, Loss: 0.6811718940734863
Epoch 7, Loss: 0.4822832942008972
Epoch 8, Loss: 0.9089839458465576
Epoch 9, Loss: 0.6010143756866455
Epoch 10, Loss: 0.664049506187439
Epoch 1, Loss: 0.7566581964492798
Epoch 2, Loss: 0.6658103466033936
Epoch 3, Loss: 0.6139968633651733
Epoch 4, Loss: 0.6689863801002502
Epoch 5, Loss: 0.6054378151893616
Epoch 6, Loss: 0.6059554815292358
Epoch 7, Loss: 0.6695582866668701
Epoch 8, Loss: 0.6558816432952881
Epoch 9, Loss: 0.6339205503463745
Epoch 10, Loss: 0.6222830414772034
Attacker Baseline Model Accuracy: 66.72%
Attacker Private Model Accuracy: 66.53%


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import numpy as np

baseline_model = model
private_model = model_pr
# baseline_model.load_state_dict(torch.load('baseline_model.pth'))
# private_model.load_state_dict(torch.load('private_model.pth'))

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
baseline_model.to(device)
private_model.to(device)

# Function to train a shadow model
def train_shadow_model(data_loader, model, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(epochs):
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

# Create shadow datasets
def create_shadow_datasets(train_dataset, num_shadow_models=5):
    shadow_datasets = []
    size = len(train_dataset) // num_shadow_models
    for _ in range(num_shadow_models):
        indices = torch.randperm(len(train_dataset))[:size]
        shadow_data = torch.utils.data.Subset(train_dataset, indices)
        shadow_datasets.append(shadow_data)
    return shadow_datasets

# Train shadow models
num_shadow_models = 5
shadow_datasets = create_shadow_datasets(train_dataset, num_shadow_models)

shadow_models_baseline = [CIFAR10Classifier().to(device) for _ in range(num_shadow_models)]
# shadow_models_private = [CIFAR10Classifier().to(device) for _ in range(num_shadow_models)]
criterion = nn.CrossEntropyLoss()

for model in shadow_models_baseline:
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    shadow_loader = DataLoader(shadow_datasets.pop(), batch_size=64, shuffle=True)
    train_shadow_model(shadow_loader, model, optimizer, criterion)

shadow_datasets_private = create_shadow_datasets(train_dataset, num_shadow_models)
shadow_models_private = [CIFAR10Classifier().to(device) for _ in range(num_shadow_models)]

for model, shadow_data in zip(shadow_models_private, shadow_datasets_private):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    shadow_loader = DataLoader(shadow_data, batch_size=64, shuffle=True)
    train_shadow_model(shadow_loader, model, optimizer, criterion)

# Function to get outputs and labels from shadow models
def get_shadow_outputs_and_labels(models, loader, label):
    all_outputs = []
    all_labels = []
    for model in models:
        model.eval()
        outputs = []
        labels = []
        with torch.no_grad():
            for data, _ in loader:
                data = data.to(device)
                output = model(data)
                outputs.append(output.cpu().numpy())
                labels.extend([label] * data.size(0))
        outputs = np.concatenate(outputs, axis=0)
        all_outputs.append(outputs)
        all_labels.extend(labels)
    all_outputs = np.concatenate(all_outputs, axis=0)
    return all_outputs, all_labels



Epoch 1, Loss: 0.5297827124595642
Epoch 2, Loss: 0.667941153049469
Epoch 3, Loss: 0.5147463083267212
Epoch 4, Loss: 0.6013748049736023
Epoch 5, Loss: 0.6876029372215271
Epoch 6, Loss: 0.6450225710868835
Epoch 7, Loss: 0.6657246351242065
Epoch 8, Loss: 0.5786383748054504
Epoch 9, Loss: 0.6009915471076965
Epoch 10, Loss: 0.6212524771690369
Epoch 1, Loss: 0.6416228413581848
Epoch 2, Loss: 0.6235588192939758
Epoch 3, Loss: 0.5773477554321289
Epoch 4, Loss: 0.5311208367347717
Epoch 5, Loss: 0.5373037457466125
Epoch 6, Loss: 0.5162485837936401
Epoch 7, Loss: 0.6883094310760498
Epoch 8, Loss: 0.6218485236167908
Epoch 9, Loss: 0.6208744049072266
Epoch 10, Loss: 0.6434410810470581
Attacker Baseline Model Accuracy: 66.67%
Attacker Private Model Accuracy: 66.89%


In [ ]:
# Generate membership labels and outputs for attacker models using shadow models
# Seen data: label 1
seen_outputs_shadow_baseline, seen_labels_shadow_baseline = get_shadow_outputs_and_labels(shadow_models_baseline, train_loader, 1)
seen_outputs_shadow_private, seen_labels_shadow_private = get_shadow_outputs_and_labels(shadow_models_private, train_loader, 1)

# Unseen data: label 0
unseen_outputs_shadow_baseline, unseen_labels_shadow_baseline = get_shadow_outputs_and_labels(shadow_models_baseline, val_loader, 0)
unseen_outputs_shadow_baseline_test, unseen_labels_shadow_baseline_test = get_shadow_outputs_and_labels(shadow_models_baseline, test_loader, 0)

unseen_outputs_shadow_private, unseen_labels_shadow_private = get_shadow_outputs_and_labels(shadow_models_private, val_loader, 0)
unseen_outputs_shadow_private_test, unseen_labels_shadow_private_test = get_shadow_outputs_and_labels(shadow_models_private, test_loader, 0)

# Combine seen and unseen data for shadow models
baseline_outputs_shadow = np.vstack([seen_outputs_shadow_baseline, unseen_outputs_shadow_baseline, unseen_outputs_shadow_baseline_test])
baseline_labels_shadow = seen_labels_shadow_baseline + unseen_labels_shadow_baseline + unseen_labels_shadow_baseline_test
private_outputs_shadow = np.vstack([seen_outputs_shadow_private, unseen_outputs_shadow_private, unseen_outputs_shadow_private_test])
private_labels_shadow = seen_labels_shadow_private + unseen_labels_shadow_private + unseen_labels_shadow_private_test

# Define and train attacker models using shadow model data
class ImprovedAttackerModel(nn.Module):
    def __init__(self):
        super(ImprovedAttackerModel, self).__init__()
        self.fc1 = nn.Linear(10, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

attacker_baseline = ImprovedAttackerModel().to(device)
attacker_private = ImprovedAttackerModel().to(device)

criterion = nn.CrossEntropyLoss()
optimizer_attacker_baseline = optim.Adam(attacker_baseline.parameters(), lr=0.001)
optimizer_attacker_private = optim.Adam(attacker_private.parameters(), lr=0.001)

def train_attacker(model, data, labels, optimizer, epochs=10):
    model.train()
    data = torch.tensor(data, dtype=torch.float32).to(device)
    labels = torch.tensor(labels, dtype=torch.long).to(device)
    dataset = torch.utils.data.TensorDataset(data, labels)
    loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

    for epoch in range(epochs):
        for inputs, targets in loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

train_attacker(attacker_baseline, baseline_outputs_shadow, baseline_labels_shadow, optimizer_attacker_baseline)
train_attacker(attacker_private, private_outputs_shadow, private_labels_shadow, optimizer_attacker_private)

# Evaluate attacker models
def evaluate_attacker(model, data, labels):
    model.eval()
    data = torch.tensor(data, dtype=torch.float32).to(device)
    labels = torch.tensor(labels, dtype=torch.long).to(device)
    dataset = torch.utils.data.TensorDataset(data, labels)
    loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=False)

    correct = 0
    with torch.no_grad():
        for inputs, targets in loader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == targets).item()
    accuracy = correct / len(labels)
    return accuracy

attacker_baseline_acc = evaluate_attacker(attacker_baseline, baseline_outputs_shadow, baseline_labels_shadow)
attacker_private_acc =  evaluate_attacker(attacker_private, private_outputs, private_labels)

print(f'Attacker Baseline Model Accuracy: {attacker_baseline_acc * 100:.2f}%')
print(f'Attacker Private Model Accuracy: {attacker_private_acc * 100:.2f}%')



In [ ]:
class ImprovedAttackerModel(nn.Module):
    def __init__(self):
        super(ImprovedAttackerModel, self).__init__()
        self.fc1 = nn.Linear(10, 256)
        self.bn1 = nn.BatchNorm1d(256)  # Add BatchNorm layer
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)  # Add BatchNorm layer
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
attacker_baseline = ImprovedAttackerModel().to(device)
attacker_private = ImprovedAttackerModel().to(device)

criterion = nn.CrossEntropyLoss()
optimizer_attacker_baseline = optim.Adam(attacker_baseline.parameters(), lr=0.001)
optimizer_attacker_private = optim.Adam(attacker_private.parameters(), lr=0.001)

def train_attacker(model, data, labels, optimizer, epochs=10):
    model.train()
    data = torch.tensor(data, dtype=torch.float32).to(device)
    labels = torch.tensor(labels, dtype=torch.long).to(device)
    dataset = torch.utils.data.TensorDataset(data, labels)
    loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

    for epoch in range(epochs):
        for inputs, targets in loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

train_attacker(attacker_baseline, baseline_outputs_shadow, baseline_labels_shadow, optimizer_attacker_baseline)
train_attacker(attacker_private, private_outputs_shadow, private_labels_shadow, optimizer_attacker_private)

# Evaluate attacker models
def evaluate_attacker(model, data, labels):
    model.eval()
    data = torch.tensor(data, dtype=torch.float32).to(device)
    labels = torch.tensor(labels, dtype=torch.long).to(device)
    dataset = torch.utils.data.TensorDataset(data, labels)
    loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=False)

    correct = 0
    with torch.no_grad():
        for inputs, targets in loader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == targets).item()
    accuracy = correct / len(labels)
    return accuracy

attacker_baseline_acc = evaluate_attacker(attacker_baseline, baseline_outputs_shadow, baseline_labels_shadow)
attacker_private_acc =  evaluate_attacker(attacker_private, private_outputs, private_labels)

print(f'Attacker Baseline Model Accuracy: {attacker_baseline_acc * 100:.2f}%')
print(f'Attacker Private Model Accuracy: {attacker_private_acc * 100:.2f}%')

Epoch 1, Loss: 0.6331300139427185
Epoch 2, Loss: 0.7069498896598816
Epoch 3, Loss: 0.6448447704315186
Epoch 4, Loss: 0.7074538469314575
Epoch 5, Loss: 0.6209133863449097
Epoch 6, Loss: 0.6248080134391785
Epoch 7, Loss: 0.5333105325698853
Epoch 8, Loss: 0.7056225538253784
Epoch 9, Loss: 0.641893208026886
Epoch 10, Loss: 0.5518613457679749
Epoch 1, Loss: 0.6261704564094543
Epoch 2, Loss: 0.6842180490493774
Epoch 3, Loss: 0.5571067333221436
Epoch 4, Loss: 0.6878211498260498
Epoch 5, Loss: 0.6012611389160156
Epoch 6, Loss: 0.7103956937789917
Epoch 7, Loss: 0.6223320960998535
Epoch 8, Loss: 0.7258370518684387
Epoch 9, Loss: 0.6210140585899353
Epoch 10, Loss: 0.7100357413291931
Attacker Baseline Model Accuracy: 66.67%
Attacker Private Model Accuracy: 66.89%


In [ ]:
torch.save(attacker_baseline.state_dict(),os.path.join(path, 'main_attacker2.pth'))
torch.save(attacker_private.state_dict(),os.path.join(path, 'private_attacker2.pth'))

In [ ]:
baseatt = ImprovedAttackerModel()
priatt = ImprovedAttackerModel()
baseatt.load_state_dict(torch.load(os.path.join(path, 'main_attacker2.pth')))
priatt.load_state_dict(torch.load(os.path.join(path, 'private_attacker2.pth')))

<All keys matched successfully>